IMPORTAR LIBRERIAS

In [2]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

# Descripción de las Columnas del Archivo de Datos Transaccionales

El archivo contiene datos transaccionales con las siguientes columnas:

- **ID de la orden (`order_id`)**: Identificador único para cada transacción.
- **ID del usuario (`user_id`)**: Identifica al cliente que realizó la compra.
- **Número de orden (`order_number`)**: Indica el número de la orden en el historial del cliente.
- **Día de la semana (`order_dow`)**: Día de la semana en que se realizó la orden.
- **Hora del día (`order_hour_of_day`)**: Hora exacta de la transacción.
- **Días desde la última orden (`days_since_prior_order`)**: Tiempo desde la última compra realizada por el cliente.
- **ID y Nombre del producto (`product_id` y `product_name`)**: Identificadores y nombres de los productos comprados.
- **Orden en el carrito (`add_to_cart_order`)**: Secuencia en la que los productos fueron añadidos al carrito.
- **Producto reordenado (`reordered`)**: Indica si el producto fue comprado anteriormente (1) o no (0).
- **ID y Nombre del Departamento (`department_id` y `department`)**: Identificador y categoría general a la que pertenece cada producto.

### Carga de Datos

In [4]:
df = pd.read_csv('media/Comercio Electrónico Evaluación.csv')
df.head(10)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables
5,2425083,49125,1,2,18,NaN,91,6,0,16,dairy eggs,soy lactosefree
6,2425083,49125,1,2,18,NaN,120,7,0,16,dairy eggs,yogurt
7,2425083,49125,1,2,18,NaN,59,8,0,15,canned goods,canned meals beans
8,2425083,49125,1,2,18,NaN,35,9,0,12,meat seafood,poultry counter
9,1944304,162867,1,3,17,NaN,37,1,0,1,frozen,ice cream ice


### 1. Analisis Exploratorio

In [5]:
print("Dimensiones del dataset:", df.shape)

Dimensiones del dataset: (2019501, 12)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019501 entries, 0 to 2019500
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   order_dow               int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   department_id           int64  
 10  department              object 
 11  product_name            object 
dtypes: float64(1), int64(9), object(2)
memory usage: 184.9+ MB


In [7]:
#Estadistica descriptiva
print("Estadistica descriptiva")
df.describe()

Estadistica descriptiva


,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id
count,2.019501e+06,2.019501e+06,2.019501e+06,2.019501e+06,2.019501e+06,1.895159e+06,2.019501e+06,2.019501e+06,2.019501e+06,2.019501e+06
mean,1.707013e+06,1.030673e+05,1.715138e+01,2.735367e+00,1.343948e+01,1.138603e+01,7.120590e+01,8.363173e+00,5.897427e-01,9.928349e+00
std,9.859832e+05,5.949117e+04,1.752576e+01,2.093882e+00,4.241008e+00,8.970980e+00,3.820727e+01,7.150059e+00,4.918804e-01,6.282933e+00
min,1.000000e+01,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
25%,8.526490e+05,5.158400e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,3.100000e+01,3.000000e+00,0.000000e+00,4.000000e+00
50%,1.705004e+06,1.026900e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,8.300000e+01,6.000000e+00,1.000000e+00,9.000000e+00
75%,2.559031e+06,1.546000e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01,1.070000e+02,1.100000e+01,1.000000e+00,1.600000e+01
max,3.421080e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01,1.340000e+02,1.370000e+02,1.000000e+00,2.100000e+01
